In [4]:
!pip install -q -r requirements.txt

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.4 MB/s eta 0:00:00


In [ ]:
import os
import shutil
import torch
import json
import pickle
import numpy as np
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from CiteHeteroSum.CiteHeteroSum_model.Graph_Constructor import Graph, graph_construction
from CiteHeteroSum.Preprocessing.Section_Filtering import section_list, section_filtering
from CiteHeteroSum.Preprocessing.EDU_Extracting import load_model, inference as EDU_inference, extract_spans
from CiteHeteroSum.Preprocessing.EDU_Embedding import compute_word_embeddings, getPositionEncoding
from CiteHeteroSum.Preprocessing.Data2EDU_Process import EDU_process
from CiteHeteroSum.Preprocessing.EDU_Embedding import phrase_embedding
from CiteHeteroSum.Preprocessing.Doc_Cite_Merging import merge_citation_to_doc_section, citation_filtering
from CiteHeteroSum.utils import *
from config import args

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

## EDU Process

In [7]:
def embedding(inp):
    inp["label_embedding"] = phrase_embedding(inp["label"][0])
    for sec_pos, secID in enumerate(inp["section_list"]):
        sec = inp["units"][secID]

        all_sent_embeddings = []
        for sent_pos, sentID in enumerate(sec["children"]):
            sent = inp["units"][sentID]

            all_edu_embeddings = []
            for edu_pos, eduID in enumerate(sent["children"]):
                edu = inp["units"][eduID]
                edu["embedding"] = phrase_embedding(edu["text"]) + getPositionEncoding(sec_pos) + getPositionEncoding(sent_pos) + getPositionEncoding(edu_pos)
                if edu["embedding"] is not None:
                    all_edu_embeddings.append(edu["embedding"])
            sent["embedding"] = torch.stack(all_edu_embeddings).mean(dim=0)
            all_sent_embeddings.append(sent["embedding"])
        sec["embedding"] = torch.stack(all_sent_embeddings).mean(dim=0)
    return inp

In [8]:
def sim_caculating(inp):
    sec_text_dict = dict()

    for sec_pos, secID in enumerate(inp["document"]["section_list"]):
        sec = inp["document"]["units"][secID]
        sec_text = ""
        for sent_pos, sentID in enumerate(sec["children"]):
            sent = inp["document"]["units"][sentID]
            sec_text = sec_text + " " + sent["text"]
        sec_text_dict[secID] = sec_text

    for sec_pos, secID in enumerate(inp["citation"]["section_list"]):
        sec = inp["citation"]["units"][secID]
        for sent_pos, sentID in enumerate(sec["children"]):
            sent = inp["citation"]["units"][sentID]
            sent["cite"] = {
                "cosine": dict(),
                "ltoken": len(sent["text"].split(" ")),
                "r2p": dict()
            }

            for tmpsecID in sec_text_dict:
                sent_text = sent["text"]
                section_text = sec_text_dict[tmpsecID]
                sent["cite"]["cosine"][tmpsecID] =  get_cosine_similarity(sent_text, section_text)
                sent["cite"]["r2p"][tmpsecID] = r2p(get_rouge(section_text, sent_text))

            for edu_pos, eduID in enumerate(sent["children"]):
                edu = inp["citation"]["units"][eduID]
                edu["cite"] = {
                "cosine": dict(),
                "ltoken": len(edu["text"].split(" ")),
                "r2p": dict()
                }
                for  tmpsecID in sec_text_dict:
                    edu_text = edu["text"]
                    section_text = sec_text_dict[tmpsecID]
                    edu["cite"]["cosine"][tmpsecID] =  get_cosine_similarity(edu_text, section_text)
                    edu["cite"]["r2p"][tmpsecID] = r2p(get_rouge(section_text, edu_text))

    return inp

In [9]:
def oracle_caculating(inp):
    label = inp["label"][0]

    for typee in ["document", "citation"]:
        for unitID, unit in inp[typee]["units"].items():
            if unit["unit_type"] != "edu":
                continue

            golden_score = get_rouge(label, unit["text"])
            unit["golden_rouge"] =  {"2p": r2p(golden_score), "fmean": (r1f(golden_score) + r2f(golden_score))/2}

    return inp

## CONFIG

In [10]:
config = "Test"
scenario = "1"

valid_configs = ["Train", "Test"]
valid_scenarios = ["1", "2", "3"]

if config not in valid_configs:
    raise ValueError(f"Invalid config: {config}. Must be one of {valid_configs}")

if scenario not in valid_scenarios:
    raise ValueError(f"Invalid scenario: {scenario}. Must be one of {valid_scenarios}")

In [11]:
import json
with open("./config.json", "r") as f:
    config_data = json.load(f)

input_file_path = config_data[config][f"scenario{scenario}"]["input"]
output_file_path = config_data[config][f"scenario{scenario}"].get("output")

In [12]:
if scenario == "1":
    with open(input_file_path, "r", encoding="utf-8") as file:
        data = json.load(file)
elif scenario == "2":
    with open(input_file_path, "r", encoding="utf-8") as file:
        data = json.load(file)

    input_file_path_2 = config_data[config][f"scenario{scenario}"]["input_2"]
    with open(input_file_path_2, "r", encoding="utf-8") as file:
        data.update(json.load(file))
elif scenario == "3":
    with open(input_file_path, "rb") as f:
        data = pickle.load(f)

In [13]:
Graphs = []

for sampleID, sample in tqdm(data.items()):
    sample = EDU_process(sample)

    sample["label_embedding"] = phrase_embedding(sample["label"][0])
    sample = oracle_caculating(sample)
    sample = sim_caculating(sample)
    sample = citation_filtering(sample)
    sample = merge_citation_to_doc_section(sample, args['input_section_names'], have_label_embedding=True)
    sample = section_filtering(sample, sampleID, args['input_section_names'], args['label_rouge_threshold'])
    if len(sample["section_list"]) > 0:
        sample = embedding(sample)
        tmp_graph = graph_construction(sample, args)
        Graphs.append(tmp_graph)

100%|██████████| 62/62 [12:53<00:00, 12.47s/it]


In [15]:
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
with open(output_file_path, "wb") as f:
    pickle.dump(Graphs, f)